In [9]:
pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


**all-mpnet-base-v2** : All-round model tuned for many use-cases. Trained on a large and diverse dataset of over 1 billion training pairs.
cosine-similarity (util.cos_sim)

In [1]:
from sentence_transformers import SentenceTransformer
import json
model = SentenceTransformer('all-MiniLM-L6-v2')

vicuna

In [4]:
from sentence_transformers import SentenceTransformer, util
import json
from nltk.stem import WordNetLemmatizer
import os
import subprocess
import shlex
import spacy

# Load the lemmatizer for word normalization
lemmatizer = WordNetLemmatizer()

# Load the SentenceTransformer model
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1', device='cuda')

# Load the FAQ data
with open('/home/mane/Mane-Project/Model/faq_data.json', 'r') as file:
    data = json.load(file)

# Extract questions and answers from the data
filename = [entry['filename'] for entry in data]
context = [entry['context'] for entry in data]
questions = [entry.get('question', '') for entry in data]
answers = [entry.get('answer', '') for entry in data]

# Encode questions and answers
filename_embeddings = model.encode(filename)
context_embeddings = model.encode(context)
question_embeddings = model.encode(questions)
answer_embeddings = model.encode(answers)

# Set the working directory to the FastChat folder
fastchat_directory = '/home/mane/FastChat'
vicuna_model_path = 'lmsys/vicuna-7b-v1.5'

user_input = input("Type your question: ")
while not user_input:
    print("Please enter a valid question.")
    user_input = input("Type your question: ").strip()
    
# Initialize spaCy for lemmatization
nlp = spacy.load('en_core_web_sm')

# Define a function for lemmatization
def preprocess_text(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

def provide_document():
    return f"The document context for this question is: {filename[most_similar_index]}"
chat_history = []

def print_chat_history():
    print("\nChat History:")
    for entry in chat_history:
        print(f"{entry['user']}: {entry['message']}")
        
# Encode the user's input
user_input_embedding = model.encode(preprocess_text(user_input))

# Calculate cosine similarity between the user's input and all questions in the dataset
similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
most_similar_index = similarities.argmax()
highest_similarity = similarities[most_similar_index]

# Set the threshold values
high_threshold = 0.8
medium_threshold = 0.5

# Construct the command to run Vicuna with the user's question
# Check the similarity against different thresholds
if highest_similarity >= high_threshold:
    # High similarity, directly provide the answer
    print(f"User's input: {user_input}")
    print(f"Most similar question: {questions[most_similar_index]}")
    print(f"Corresponding answer: {answers[most_similar_index]}")
    print(f"Similarity Score: {highest_similarity}")

    # Ask for user satisfaction
    user_satisfaction = input("Is this the information you were looking for? (yes/no): ").lower()
    if user_satisfaction == 'no':
        # Continue the interaction loop
        user_input += " " + input("Please provide more details or clarify your question: ")
        user_input_embedding = model.encode(preprocess_text(user_input))
        similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
        most_similar_index = similarities.argmax()
        print(f"User's input after clarification: {user_input}")
        print(f"Most similar question after clarification: {questions[most_similar_index]}")
        print(f"Corresponding answer: {answers[most_similar_index]}")
        print(f"Similarity Score after clarification: {similarities[most_similar_index]}")

else:
    if medium_threshold <= highest_similarity < high_threshold:
        # Medium similarity, ask for clarification
        print(f"The question is similar to: {questions[most_similar_index]}")
        print(f"Similarity Score: {highest_similarity}")
        user_confirmation = input("Is this the same question? (yes/no): ").lower()

        if user_confirmation == 'yes':
            # User confirms, provide the answer
            print(f"User's input: {user_input}")
            print(f"Most similar question: {questions[most_similar_index]}")
            print(f"Corresponding answer: {answers[most_similar_index]}")
            print(f"Similarity Score: {highest_similarity}")

            # Ask for user satisfaction
            user_satisfaction = input("Is this the information you were looking for? (yes/no): ").lower()
            if user_satisfaction == 'no':
                # Continue the interaction loop
                user_input += " " + input("Please provide more details or clarify your question: ")
                user_input_embedding = model.encode(preprocess_text(user_input))
                similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
                most_similar_index = similarities.argmax()
                print(f"User's input after clarification: {user_input}")
                print(f"Most similar question after clarification: {questions[most_similar_index]}")
                print(f"Corresponding answer: {answers[most_similar_index]}")
                print(f"Similarity Score after clarification: {similarities[most_similar_index]}")
        else:
            # User denies, ask for clarification
            user_input += " " + input("Please provide more details or clarify your question: ")
            user_input_embedding = model.encode(preprocess_text(user_input))
            similarities = util.cos_sim(user_input_embedding, question_embeddings)[0]
            most_similar_index = similarities.argmax()
            print(f"User's input after clarification: {user_input}")
            print(f"Most similar question after clarification: {questions[most_similar_index]}")
            print(f"Corresponding answer: {answers[most_similar_index]}")
            print(f"Similarity Score after clarification: {similarities[most_similar_index]}")
    else:
        # Low similarity, use Vicuna for answering
        print("Low similarity. Asking Vicuna for an answer...")

        # Set the working directory to the FastChat folder
        os.chdir('/home/mane/FastChat')

        # Construct the command to run Vicuna with the user's question
        vicuna_command = ['python3', '-m', 'fastchat.serve.cli', '--model-path', f'lmsys/vicuna-7b-v1.5']
        try:
            # Use subprocess to run Vicuna and capture its output
            vicuna_process = subprocess.Popen(vicuna_command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            vicuna_out, vicuna_err = vicuna_process.communicate(input=user_input)
            
            if vicuna_process.returncode == 0:
                # Extract user's question and Vicuna's answer from the output
                lines = vicuna_out.strip().split('\n')
                user_question = lines[0].split('USER: ')[-1].strip()
                vicuna_answer = lines[-1].split('ASSISTANT: ')[-1].strip()
                
                # Display the extracted information in the desired format
                print(f"user: {user_question}")
                print(f"chatbot: {vicuna_answer}")
            else:
                print(f"Error while running Vicuna: {vicuna_err}")
        except Exception as e:
            print(f"Error while running Vicuna: {e}")
            vicuna_answer = "Error while running Vicuna"


Low similarity. Asking Vicuna for an answer...
user: ASSISTANT: Hello! As an AI language model, I don't have emotions or experiences, but I'm here to assist you with any questions or tasks you may have. How can I help you today?
chatbot: USER: exit...
